In [1]:
# Feistel cipher with SHA-256
import hashlib

def feistel_encrypt_block(key:bytes, block:bytes, rounds=16):
    left = block[:len(block)//2]
    right = block[len(block)//2:]
    for i in range(rounds):
        f = hashlib.sha256(right + key + bytes(i)).digest()[:len(right)]
        new_right = bytes([a ^ b for a, b in zip(f, left)])
        left, right = right, new_right
    return left + right

def feistel_decrypt_block(key:bytes, block:bytes, rounds=16):
    left = block[:len(block)//2]
    right = block[len(block)//2:]
    for i in range(rounds-1, -1, -1):
        f = hashlib.sha256(left + key + bytes(i)).digest()[:len(right)]
        new_left = bytes([a ^ b for a, b in zip(f, right)])
        right, left = left, new_left
    return left + right

cipher = feistel_encrypt_block(b'monkey', b"bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")
feistel_decrypt_block(b'monkey', cipher)

b'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb'

In [2]:
# DES: Data Encryption Standard (1977)

from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad # PKCS7 padding (default)

# Key must be 8 bytes long
key = b'8bytekey'

# Create a DES cipher object in ECB mode
cipher = DES.new(key, DES.MODE_ECB)
message = b'Secret message'

# Pad the message to be a multiple of 8 bytes
padded_message = pad(message, DES.block_size)

# Encrypt the message
encrypted_message = cipher.encrypt(padded_message)
print(f'Encrypted: {encrypted_message}')

# Decrypt the message
decrypted_message = cipher.decrypt(encrypted_message)

# Unpad the decrypted message
unpadded_message = unpad(decrypted_message, DES.block_size)
print(f'Decrypted: {unpadded_message}')

Encrypted: b'nX\xb4\xff~\xe5\xb2Gmz\xb9\x0e_\x92ov'
Decrypted: b'Secret message'


In [4]:
from Crypto.Cipher import AES

# Key must be 16, 24, or 32 bytes long
aes_key = b'16bytekey1234567'

# Create an AES cipher object in ECB mode
aes_cipher = AES.new(aes_key, AES.MODE_ECB)
aes_message = b'Secret AES message'

# Pad the message to be a multiple of 16 bytes
aes_padded_message = pad(aes_message, AES.block_size)

# Encrypt the message
aes_encrypted_message = aes_cipher.encrypt(aes_padded_message)
print(f'Encrypted (AES): {aes_encrypted_message}')

# Decrypt the message
aes_decrypted_message = aes_cipher.decrypt(aes_encrypted_message)

# Unpad the decrypted message
aes_unpadded_message = unpad(aes_decrypted_message, AES.block_size)
print(f'Decrypted (AES): {aes_unpadded_message}')

Encrypted (AES): b"\xa2\x12\x8b\x86\x03\xd1\xb2;\x93\xcb\n\x0c\xbd\xee\xaf\xd5\xc6\xde'e<\xef\x06\x17\xaa\x10\x10\xf7\x84\xab\x16\xa3"
Decrypted (AES): b'Secret AES message'


In [42]:
# Encrypt with counter mode

from Crypto.Util import Counter

# Create a counter object for CTR mode
ctr = Counter.new(128)
aes_key = b'16bytekey1122393'
aes_ctr_cipher = AES.new(aes_key, AES.MODE_CTR, counter=ctr)

# Encrypt the message
aes_message = b'Super secret information'
print(f'Message: {aes_message}')
aes_ctr_encrypted_message = aes_ctr_cipher.encrypt(aes_message)
print(f'Encrypted (AES CTR): {aes_ctr_encrypted_message}')


# Sent over the internet (any issues?)


# Decrypt the message
ctr = Counter.new(128)
aes_ctr_decipher = AES.new(aes_key, AES.MODE_CTR, counter=ctr)
aes_ctr_decrypted_message = aes_ctr_decipher.decrypt(aes_ctr_encrypted_message)
print(f'Decrypted (AES CTR): {aes_ctr_decrypted_message}')

Message: b'Super secret information'
Encrypted (AES CTR): b'\xa0\xc2#s\xd7\x9e\xf3Ke\x03\xf3\n\xabL t\x17\xd0\x96\xb94mlm'
Decrypted (AES CTR): b'Super secret information'


In [49]:
# MAC (Message Authentication Code)

import hmac
import hashlib
from Crypto.Util import Counter
from Crypto.Cipher import AES

# Shared secret key
shared_key = b'shared_secretkey'

# Alice creates a message and its MAC
message = b'This is a secret message'
print(f'Alice\'s Message: {message.decode()}')
alice_mac = hmac.new(shared_key, message, hashlib.sha256).digest()
ctr = Counter.new(128)
aes_cipher = AES.new(shared_key, AES.MODE_CTR, counter=ctr)
alice_cipher = aes_cipher.encrypt(message)
print(f'Encrypted (AES CTR): {alice_cipher.hex()}')
print(f'MAC: {alice_mac.hex()}')


# Data is sent through the network
cipher = alice_cipher
mac = alice_mac
print(cipher.count(b'\xfd'))
cipher = cipher.replace(b'\xfd', b'\xfe')

# Bob receives the message and verifies the MAC
ctr = Counter.new(128)
aes_decipher = AES.new(shared_key, AES.MODE_CTR, counter=ctr)
decrypted_message = aes_decipher.decrypt(cipher)
print(f'Decrypted (AES CTR): {decrypted_message}')
bob_mac = hmac.new(shared_key, decrypted_message, hashlib.sha256).digest()
if hmac.compare_digest(bob_mac, mac):
    print("MAC verified.")
else:
    print("MAC verification failed.")



Alice's Message: This is a secret message
Encrypted (AES CTR): e05e278dfd81925147bbaf144c8a81b4d220f51943edde0c
MAC: a4eacc6f10ab37b6a4f5fb25539659f7a09ff73627fc7a7694661ff36c1a5cf8
1
Decrypted (AES CTR): b'This#is a secret message'
MAC verification failed.
